In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.sparse as sp
import scipy.stats
import os
import scipy.io as sio
#import dnatools
#import re
import regex as re
from collections import Counter, defaultdict
#from pylab import *
#import matplotlib.pyplot as plt
import sys 
#%matplotlib inline

def distance(astring, bstring) :
    distance = 0
    
    limit = len(astring)
    diff = len(bstring) - len(astring)
    if len(bstring) < len(astring) :
        limit = len(bstring)
        diff = len(astring) - len(bstring)
    
    for i in range(limit) :
        if astring[i] != bstring[i] :
            distance += 1
    return distance + diff

print(distance("ATATATGCGCGC", "TTATATCCGCAC"))

3


C:\Users\Johannes\Anaconda3\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
#dna_file = pd.read_csv('apa_nextseq_v2_dna_filtered_20160623.csv',sep=',')
dna_file = pd.read_csv('apa_nextseq_v2_dna_merged_20160922.csv',sep=',')

dna_upstream_list = list(dna_file.upstream_seq)
dna_pas_list = list(dna_file.pas_seq)
dna_downstream_list = list(dna_file.downstream_seq)
dna_seq_list = list(dna_file.seq)
dna_library_list = list(dna_file.library)

dna_upstream_map = {}
dna_pas_map = {}
dna_downstream_map = {}
dna_seq_map = {}
dna_library_map = {}

for i in range(0, len(dna_upstream_list)) :
    dna_upstream_map[dna_upstream_list[i]] = dna_upstream_list[i]
    dna_pas_map[dna_upstream_list[i]] = dna_pas_list[i]
    dna_downstream_map[dna_upstream_list[i]] = dna_downstream_list[i]
    dna_seq_map[dna_upstream_list[i]] = dna_seq_list[i]
    dna_library_map[dna_upstream_list[i]] = int(dna_library_list[i])

n10_a = re.compile(r"(TGTTAAGAAC[ACGTN]{10}CTGGTAACTGACCTTCAAAG){s<=3}")
n10_b = re.compile(r"(TGTTAAGAACAAGTT[ACGTN]{10}AACTGACCTTCAAAG){s<=3}")
n10_c = re.compile(r"(TGTTAAGAACAAGTTTGGCT[ACGTN]{10}ACCTTCAAAG){s<=3}")

wt20_down = re.compile(r"(GATGTCTCGTGATCTGGTGT){s<=3}")

In [3]:
#Library bias splits
lib_a = re.compile(r"([ACGTN]{20}CTGGTAACTGACCTTCAAAG){s<=3}")
lib_b = re.compile(r"(TGTTAAGAAC[ACGTN]{20}ACCTTCAAAG){s<=3}")
lib_c = re.compile(r"(TGTTAAGAACAAGTTTGGCT[ACGTN]{20}){s<=3}")
lib_d = re.compile(r"(TGTTAAGAAC[ACGTN]{10}CTGGTAACTGACCTTCAAAG){s<=3}")
lib_e = re.compile(r"(TGTTAAGAACAAGTT[ACGTN]{10}AACTGACCTTCAAAG){s<=3}")
lib_f = re.compile(r"(TGTTAAGAACAAGTTTGGCT[ACGTN]{10}ACCTTCAAAG){s<=3}")

libs = [
    [lib_d, [10,20], 1],
    [lib_e, [15,25], 1],
    [lib_f, [20,30], 1],
    [lib_a, [0, 20], 1],
    [lib_b, [10,30], 1],
    [lib_c, [20,40], 1]
]

def get_sequence_lib(seq, libs) :
    for j in range(0, len(libs)) :
        region_regex = libs[j][0]
        if re.search(region_regex, seq) :
            return j, libs[j][1][0], libs[j][1][1]
    return -1, -1, -1


def partition_lib(seqs, libs, seq_start, seq_end) :
    lib_seqs = []
    lib_mappings = []
    for j in range(0, len(libs)) :
        lib_seqs.append([])
        lib_mappings.append([])
    
    seq_i = 0
    for seq in seqs :
        inner_seq = seq[seq_start:seq_end]
        
        lib_i, start_r, end_r = get_sequence_lib(inner_seq, libs)
        if lib_i != -1 :
            lib_seqs[lib_i].append(inner_seq[start_r:end_r])
            lib_mappings[lib_i].append(seq_i)
        seq_i += 1
    return lib_seqs, lib_mappings
    
def mutate_sequences(seqs, mappings, mutate=1) :
    bases = ['A', 'C', 'G', 'T']
    mutants = {}
    
    seq_i = 0
    for seq in seqs :
        if mutate == 1 :
            for i in range(0, len(seq)) :
                for base1 in bases :
                    mutants[seq[0:i] + base1 + seq[i+1:]] = mappings[seq_i]
        elif mutate == 0 :
            mutants[seq] = mappings[seq_i]
        seq_i += 1
    return mutants

lookup_indexes = []

In [ ]:
rna_file = open('apa_nextseq_v2_rna_20160619.csv', 'r')
count = 0

mapped_count = {}
mapped_proximal_count = {}
mapped_distal_count = {}

mapped_cutsite_count = {}

mapped_umi_map = {}

total_mapped_count = 0
total_mapped_proximal_count = 0
total_mapped_distal_count = 0

bases = ['A', 'C', 'G', 'T']

dna_upstream_map_keys = list(dna_upstream_map.keys())

print('Partitioning seq data according to sublibrary bias.')

lib_seqs, lib_mappings = partition_lib(dna_upstream_map_keys, libs, 0, 40)

print('Transforming partitioned seq data to bit vector sets.')

lookup_indexes = []

for j in range(0, len(libs)) :
    seq_length = libs[j][1][1] - libs[j][1][0]
    print('Indexing sublibrary of size: ' + str(len(lib_seqs[j])))
    lookup_indexes.append(mutate_sequences(lib_seqs[j], lib_mappings[j], 0))#libs[j][2]
    #lookup_indexes.append({})

num_matched_on_dictionary_lookup = 0
num_matched_on_hamming_index = 0

print('Mapping RNA reads to DNA members.')


total_rna_lib_count = [0, 0, 0, 0, 0, 0]
total_mapped_lib_count = [0, 0, 0, 0, 0, 0]


rna_file.readline()

i = 0

for k in range(1):
    for line in rna_file :
    #while True:
        #line = rna_file.readline()
        i += 1
        if line == None :
            break
        
        lineparts = line.split(',')
        if len(lineparts) != 6 :
            break
        
        upstream_seq = lineparts[0][10:50]
        downstream_seq = lineparts[1]
        if downstream_seq != '' :
            downstream_seq = downstream_seq[10:30]
        
        umi_seq = lineparts[3]
        cut_pos = int(lineparts[4])
        is_proximal = int(lineparts[5])
        
        lib_i, start_r, end_r = get_sequence_lib(upstream_seq, libs)
        
        dna_upstream_seq = None
        
        if lib_i == 3 or lib_i == 4 or lib_i == 5 :
            
            rna_library = lib_i
            if downstream_seq != '' :
                if re.search(wt20_down, downstream_seq) != None :
                    rna_library = lib_i - 3
                total_rna_lib_count[rna_library] += 1
            
            if upstream_seq in dna_upstream_map :
                dna_upstream_seq = upstream_seq
                num_matched_on_dictionary_lookup += 1
            elif lib_i != -1 and upstream_seq[start_r:end_r] in lookup_indexes[lib_i] :
                mapped_upstream_i = lookup_indexes[lib_i][upstream_seq[start_r:end_r]]
                dna_upstream_seq = dna_upstream_map_keys[mapped_upstream_i]
                num_matched_on_hamming_index += 1
            
            if dna_upstream_seq != None :
                if distance(upstream_seq[start_r:end_r], dna_upstream_seq[start_r:end_r]) > 1 :
                    print("ERROR!")
                    break
            
                total_mapped_lib_count[dna_library_map[dna_upstream_seq]] += 1
            
                dna_downstream_seq = dna_downstream_map[dna_upstream_seq]
            
                if dna_upstream_seq not in mapped_count :
                    mapped_count[dna_upstream_seq] = 0
                    mapped_proximal_count[dna_upstream_seq] = 0
                    mapped_distal_count[dna_upstream_seq] = 0
                    mapped_umi_map[dna_upstream_seq] = {}
                    mapped_cutsite_count[dna_upstream_seq] = {}
                
                is_dup_umi = False
                
                #Exact match UMI
                if umi_seq in mapped_umi_map[dna_upstream_seq] :
                    is_dup_umi = True
                else :
                    for ref_umi in mapped_umi_map[dna_upstream_seq] :
                        if distance(umi_seq, ref_umi) <= 1 :
                            is_dup_umi = True
                            mapped_umi_map[dna_upstream_seq][umi_seq] = umi_seq
                            break
                
                #if umi not in mapped_dna_umi_map[dna_seq] :
                if is_dup_umi == False :
                    mapped_umi_map[dna_upstream_seq][umi_seq] = True
                    
                    if is_proximal == 1 :
                        #Proximal
                        mapped_proximal_count[dna_upstream_seq] += 1
                        mapped_count[dna_upstream_seq] += 1
                        total_mapped_proximal_count += 1
                        total_mapped_count += 1
                        
                        if cut_pos not in mapped_cutsite_count[dna_upstream_seq] :
                            mapped_cutsite_count[dna_upstream_seq][cut_pos] = 0
                        mapped_cutsite_count[dna_upstream_seq][cut_pos] += 1
                    elif downstream_seq != '' and distance(downstream_seq, dna_downstream_seq) <= 3 :
                        #Distal
                        mapped_distal_count[dna_upstream_seq] += 1
                        mapped_count[dna_upstream_seq] += 1
                        total_mapped_distal_count += 1
                        total_mapped_count += 1
        
        if count % 1000000 == 0:
            print('Count: ' + str(count))
            print('Matched on Dict lookup: ' + str(num_matched_on_dictionary_lookup))
            print('Matched on Hamming index: ' + str(num_matched_on_hamming_index))
            
            print(str(len(mapped_count)) + ' mapped unique library members')
            
            print(str(total_mapped_proximal_count) + ' mapped proximal RNA reads')
            print(str(total_mapped_distal_count) + ' mapped distal RNA reads')
            
            print('Extracted RNA library distribution: ' + str(total_rna_lib_count))
            print('Mapped library distribution: ' + str(total_mapped_lib_count))
        count += 1
    
print('COMPLETE')
print(str(len(mapped_count)) + ' mapped unique library members')
print(str(total_mapped_count) + ' mapped RNA reads')
print(str(total_mapped_proximal_count) + ' mapped proximal RNA reads')
print(str(total_mapped_distal_count) + ' mapped distal RNA reads')

print('Extracted RNA library distribution: ' + str(total_rna_lib_count))
print('Mapped library distribution: ' + str(total_mapped_lib_count))

rna_file.close()

mapped_upstream_list = []
mapped_pas_list = []
mapped_downstream_list = []
mapped_seq_list = []
mapped_proximal_list = []
mapped_distal_list = []
mapped_count_list = []
mapped_library_list = []

lib_size = 0
for upstream_seq in mapped_count :
    if mapped_count[upstream_seq] > 0 :
        lib_size += 1
cut_count_matrix = sp.lil_matrix((lib_size, 200))

i = 0
for upstream_seq in mapped_count :
    if mapped_count[upstream_seq] > 0 :
        mapped_upstream_list.append(upstream_seq)
        mapped_pas_list.append(dna_pas_map[upstream_seq])
        mapped_downstream_list.append(dna_downstream_map[upstream_seq])
        mapped_seq_list.append(dna_seq_map[upstream_seq])
        mapped_proximal_list.append(mapped_proximal_count[upstream_seq])
        mapped_distal_list.append(mapped_distal_count[upstream_seq])
        mapped_count_list.append(mapped_count[upstream_seq])
        mapped_library_list.append(dna_library_map[upstream_seq])
        
        for cut_pos in mapped_cutsite_count[upstream_seq] :
            cut_count = mapped_cutsite_count[upstream_seq][cut_pos]
            cut_count_matrix[i, cut_pos] = float(cut_count)
        i += 1
            
cut_count_matrix = sp.csr_matrix(cut_count_matrix)
cut_dict = dict()
cut_dict["cuts"] = cut_count_matrix
sio.savemat('apa_nextseq_v2_library_merged2_20161001', cut_dict)

print(str(len(mapped_upstream_list)) + ' kept library members')

df = pd.DataFrame({'upstream'  : mapped_upstream_list,
                   'pas'  : mapped_pas_list,
                   'downstream'  : mapped_downstream_list,
                   'seq'  : mapped_seq_list,
                   'library' : mapped_library_list,
                   'proximal_count'  : mapped_proximal_list,
                   'distal_count' : mapped_distal_list,
                   'total_count' : mapped_count_list})

df = df.sort_values('total_count')

print(len(df))
#df = df.drop_duplicates() # This should drop umi copies, but will miss umi matches with different cut sites
#print(len(df))
new_columns = ['upstream', 'pas', 'downstream', 'seq', 'library', 'proximal_count', 'distal_count', 'total_count']
df.to_csv('apa_nextseq_v2_library_merged2_20161001.csv', index=False)
